In [ ]:
import pandas as pd
import json
import helpers.functions as h
import numpy as np

In [ ]:
# Load assets
with open('config_test.json') as file:
    asset_names = json.load(file)

In [ ]:
# User selects assets for direct comparison.
COMPARISON = ['SQQQ',
              'JDST',
              'FAZ',
              'SPXS',
              'SDOW']

In [ ]:
# Initialise datframe to log modeling params
asset_modeling = pd.DataFrame(columns = [
                                    'ticker',
                                    'slope',      
                                    'intercept',       
                                    'r_value',     
                                    'p_value',  
                                    'std_err'
                                    ])

# Iterate over assets, retrive data and make calculations.

# Underlaying assets
for count, index in enumerate(asset_names["indices"]):
    # Derivatives of underlaying assets
    for ticker in asset_names["indices"][index]["Derivatives"]:
        print(f'retrieving {ticker} data')

        # Retrive price data
        df = h.getData(ticker)

        # Check dataframe is populated
        if len(df) > 0:
            # Plot price and logged price vs time 
            h.price_vs_time(df, ticker, log = False)

            # As above but return fitting params
            slope, intercept, r_value, p_value, std_err = h.price_vs_time(df, ticker, log = True)
            
            # Add fitting params to dataframe
            temp_df = pd.DataFrame(data = {
                                        'ticker': ticker,
                                        'slope': slope, 
                                        'intercept': intercept,     
                                        'r_value': r_value,       
                                        'p_value': p_value,     
                                        'std_err': std_err
                                        },  index=[0])
            
            asset_modeling = pd.concat([asset_modeling, temp_df])

            # Calculate largest peak to trough drawdown in dataframe
            drawdown_df = h.largestDrawDown(df)

            # Plot daily price movement distribution.
            daily_distribtion = h.distribution_plot(df, ticker)

            # Aggregate to compare accross assets from user selection
            if ticker in COMPARISON:
                sm_df = df[['Date', 'Log Close']]
                try:
                    comparison_df = h.composite_dataframe(comparison_df, sm_df, ticker)

                except:
                    comparison_df = sm_df.rename(columns={"Log Close": f"{ticker}_Log_Close"})
                
        else:
            print('Dataframe Empty')

h.comparison_plot(comparison_df)
h.linear_model_plot(asset_modeling)